In [ ]:
from pathlib import Path
import pandas as pd
from rdflib import Graph, URIRef
from rdflib.namespace import OWL, RDFS, XSD

# Start from the notebook's current directory
NB_DIR = Path.cwd().resolve()

# Walk up to find a directory that contains a 'src' folder
SRC_DIR = None
for p in [NB_DIR] + list(NB_DIR.parents):
    candidate = p / "assignment" / "src"
    if candidate.exists():
        SRC_DIR = candidate
        break
    candidate = p / "src"  # also allow projects that don't have the extra 'assignment' level
    if candidate.exists():
        SRC_DIR = candidate
        break

if SRC_DIR is None:
    raise FileNotFoundError(
        f"Could not find a 'src' directory above {NB_DIR}. "
        "If your project root is at e.g. .../assignment/, open the notebook there or set SRC_DIR manually."
    )

DATA_DIR = SRC_DIR / "data"

print("Notebook CWD:   ", NB_DIR)
print("Resolved SRC_DIR:", SRC_DIR)
print("Resolved DATA_DIR:", DATA_DIR)
print("TTL files found:", [p.name for p in SRC_DIR.glob("*.ttl")])


TTL = {
    "bfo":  SRC_DIR / "bfo-core.ttl",
    "ies":  SRC_DIR / "ies.ttl",
    "qudt": SRC_DIR / "qudt.ttl",
    "ccom": SRC_DIR / "ccom.ttl",
    "ccot": SRC_DIR / "ccot.ttl",
    "to":   SRC_DIR / "time.ttl",
}

OUT = {
    "bfo":  DATA_DIR / "bfo-class.xlsx",
    "ies":  DATA_DIR / "ies-class.xlsx",
    "qudt": DATA_DIR / "qudt-class.xlsx",
    "ccom": DATA_DIR / "ccom-class.xlsx",
    "ccot": DATA_DIR / "ccot-class.xlsx",
    "to":   DATA_DIR / "to-class.xlsx",
}

CLASS_LABEL_QUERY = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
SELECT ?iri ?label
WHERE {
  ?iri a ?classType .
  VALUES ?classType { owl:Class rdfs:Class }
  OPTIONAL { ?iri rdfs:label ?label }

  FILTER NOT EXISTS {
    ?iri owl:deprecated ?d .
    FILTER( ?d = true || lcase(str(?d)) = "true" )
  }

  FILTER ( !isBlank(?iri) )   
}
ORDER BY ?label
"""

def run_local_select(ttl_path: Path, query: str) -> pd.DataFrame:
    g = Graph()
    g.parse(ttl_path)  # rdflib guesses Turtle from .ttl
    res = g.query(query)
    rows = []
    for row in res:
        d = row.asdict()  # {'iri': rdflib.term.URIRef(...), 'label': rdflib.term.Literal(...)} or missing
        iri_term   = d.get("iri")
        label_term = d.get("label")
        rows.append({
            "iri":   str(iri_term)   if iri_term is not None else None,
            "label": str(label_term) if label_term is not None else None,
        })
    return pd.DataFrame(rows, columns=["iri", "label"])

for key, ttl_path in TTL.items():
    if not ttl_path.exists():
        print(f"⚠️  Missing {ttl_path} — skipping {key}")
        continue
    print(f"Querying {ttl_path} …")
    df = run_local_select(ttl_path, CLASS_LABEL_QUERY)
    df.to_excel(OUT[key], index=False)
    print(f"✅ Saved {len(df)} rows to {OUT[key]}")

